In [2]:
import pymc as pm
import numpy as np
import pandas as pd
from scipy import stats

# random.seed(131)
np.random.seed(131)

# Homework 5: Part I

1. Go get data from kaggle.com and do a ***Bayesian Linear Regression*** analysis

```python
import pymc as pm; import numpy as np
n,p=100,10; X,y=np.zeros((n,p)),np.ones((n,1))
# Replace this made up data with your data set from kaggle...
with pm.Model() as MLR:
    betas = pm.MvNormal('betas', mu=np.zeros((p,1)), cov=np.eye(p), shape=(p,1))
    sigma = pm.TruncatedNormal('sigma', mu=1, sigma=1, lower=0) # half normal
    y = pm.Normal('y', mu=pm.math.dot(X, betas), sigma=sigma, observed=y)

with MLR:
    idata = pm.sample()
```    

2. Choose ***prior*** that are sensible: certainly you might change the ***hyperparameters***, and perhaps you can experiment with different distributional families for `sigma`...

3. [Optional] Assess the performance of the MCMC and note any issues or warnings

    1. Traceplots, inference (credible) intervals, effective sample sizes, energy plots, warnings and other notes... just the usual stuff they do [here](https://www.pymc.io/projects/docs/en/stable/learn/core_notebooks/pymc_overview.html#pymc-overview)

4. [Optional] Perform ***Multiple Linear Regression*** diagnostics... residual plots, etc.


In [13]:
# url = "https://raw.githubusercontent.com/ray0130/STA365/main/HW5/loan_data.csv"
url = "https://raw.githubusercontent.com/ray0130/STA365/main/HW5/honey_purity_dataset.csv"
raw_df = pd.read_csv(url)
# n,p=100,10;

df = raw_df.sample(n=1000, random_state=131)

X = df[["Density", "Viscosity", "F", "G", "Purity"]].values

y = df[["Price"]].values.reshape(-1, 1)


n, p = X.shape
# n = X.shape[0]
# p = X.shape[1]
# print(X.shape)
# print(y.shape)

# X,y=np.zeros((n,p)),np.ones((n,1))
# Replace this made up data with your data set from kaggle...
with pm.Model() as MLR:
    betas = pm.MvNormal('betas', mu=np.zeros((p,1)), cov=np.eye(p), shape=(p,1))
    sigma = pm.TruncatedNormal('sigma', mu=1, sigma=1, lower=0) # half normal
    y = pm.Normal('y', mu=pm.math.dot(X, betas), sigma=sigma, observed=y)

with MLR:
    idata = pm.sample()




# Homework 5: Part II
    
## Answer the following with respect to $p(\boldsymbol \beta |\boldsymbol\Sigma, \mathbf{X},\mathbf{y})$ on the previous slide
    
1. Rewrite $p(\boldsymbol \beta |\boldsymbol\Sigma, \mathbf{X},\mathbf{y})$ in terms of $\sigma^2$ (no longer using $\Sigma$) if $\Sigma=\sigma^2I$

\begin{align*}
p(\beta | \Sigma, X, y) = \mathcal{MVN}(E[\beta | \Sigma, X, y] = ([X^TX]^{-1} + \sigma^2\Sigma_\beta^{-1}) (X^Ty + \sigma^2\Sigma_\beta^{-1}\beta_0), Var[\beta | \Sigma, X, y] = [X^TX]^{-1} + \sigma^2\Sigma_\beta^{-1})
\end{align*}

2. What is $E[\boldsymbol \beta |\boldsymbol\Sigma, \mathbf{X},\mathbf{y}]$?

\begin{align*}
E[\beta | \Sigma, X, y] = ([X^TX]^{-1} + \sigma^2\Sigma_\beta^{-1}) (X^Ty + \sigma^2\Sigma_\beta^{-1}\beta_0)
\end{align*}

3. What ***hyperparameters*** values (legal or illegal) would make $E[\boldsymbol \beta |\boldsymbol\Sigma, \mathbf{X},\mathbf{y}] = (\mathbf{X^\top X})^{-1}\mathbf{X^\top y}$?

Observing the equation in (2), we can see that if either $\sigma^2 = 0$ or $\Sigma_\beta^{-1} = 0$ both give us the desired equation\\

\begin{align*}
\sigma^2 = 0 \quad \text{or} \quad \Sigma_\beta^{-1} = 0
\end{align*}

4. What ***hyperparameters*** values (legal or illegal) would make $E[  \mathbf{\hat y} = \mathbf{X}\boldsymbol \beta |\boldsymbol\Sigma, \mathbf{X},\mathbf{y}] = \mathbf{X}(\mathbf{X^\top X})^{-1}\mathbf{X^\top y}$?

\begin{align*}
E[  \mathbf{\hat y} &= \mathbf{X}\boldsymbol \beta |\boldsymbol\Sigma, \mathbf{X},\mathbf{y}] \\
&= X E[\hat y = \beta | \Sigma, X, y] = x([X^TX]^{-1} + \sigma^2\Sigma_{\beta}^{-1}) (X^Ty + \sigma^2\Sigma_{\beta}^{-1}\beta_0)\\
&\text{if $\sigma^2 = 0 \quad \text{or} \quad \Sigma_\beta^{-1} = 0$}\\
& = \mathbf{X}(\mathbf{X^\top X})^{-1}\mathbf{X^\top y}
\end{align*}

So the answer is the same as in (3)

5. What is $\text{Var}[\boldsymbol \beta |\boldsymbol\Sigma, \mathbf{X},\mathbf{y}]$?

\begin{align*}
Var[\beta | \Sigma, X, y] = [X^TX]^{-1} + \sigma^2\Sigma_\beta^{-1}
\end{align*}

# Homework 5: Part III

1. Go get data from kaggle.com and perform inference for a ***Bayesian Multivariate Normal Model***

<SPAN STYLE="font-size:18.0pt">

```python
import numpy as np; from scipy import stats
p=10; Psi=np.eye(p); a_cov = stats.invwishart(df=p+2, scale=Psi).rvs(1)
n=1000; y=stats.multivariate_normal(mean=np.zeros(p), cov=a_cov).rvs(size=n)
# Replace this made up data with your data set from kaggle...
    
with pm.Model() as MNV_LKJ:
    packed_L = pm.LKJCholeskyCov("packed_L", n=p, eta=2.0,
                                 sd_dist=pm.Exponential.dist(1.0, shape=2), compute_corr=False)
    L = pm.expand_packed_triangular(p, packed_L)
    # Sigma = pm.Deterministic('Sigma', L.dot(L.T)) # Don't use a covariance matrix parameterization
    mu = pm.MvNormal('mu', mu=np.array(0), cov=np.eye(p), shape=p);
    # y = pm.MvNormal('y', mu=mu, cov=Sigma, shape=(n,1), observed=y)
    # Figure out how to parameterize this with a Cholesky factor to improve computational efficiency
with MNV_LKJ    
    idata = pm.sample()
```    
</SPAN>

2. As indicated above, don't use a covariance matrix parameterization and instead figure out how to parameterize this with a ***Cholesky factor*** to improve computational efficiency. The ***Cholesky***-based formulation allows general $O(n^3)$ $\det({\boldsymbol \Sigma})$ to be computed using a simple $O(n)$ product and general $O(n^3)$ ${\boldsymbol \Sigma}^{-1}$ to be instead evaluated with $O(n^2)$ ***backward substitution***.

2. Specify ***priors*** that work: certainly you'll likely need to change the ***prior hyperparameters*** for $\boldsymbol \mu$ (`mu`) and $\mathbf{R}$ (`packed_L`)...
    1. And you could consider adjusting the ***prior*** for $\boldsymbol \sigma$ using `sd_dist`...

3. [Optional] Assess the performance of the MCMC and note any issues

    1. Traceplots, inference (credible) intervals, effective sample sizes, energy plots, warnings and other notes... just the usual stuff they do [here](https://www.pymc.io/projects/docs/en/stable/learn/core_notebooks/pymc_overview.html#pymc-overview)



In [21]:

# Using df from Part I

y = df[["Density", "Viscosity", "F", "G", "Purity"]].values
n, p = y.shape
# print(n, p, len(y))
with pm.Model() as MNV_LKJ:
    mu = pm.MvNormal('mu', mu=np.array(0), cov=np.eye(p), shape=p)
    sd_dist = pm.Exponential.dist(1.0, shape=p)
    chol, corr, sigmas  = pm.LKJCholeskyCov("chol_cov", n=p, eta=2.0,
                                 sd_dist=sd_dist, compute_corr=True)

    new_y = pm.MvNormal('new_y', mu=mu, chol=chol, observed=y)
with MNV_LKJ:
    idata = pm.sample()